In [1]:

from MyTorch import Model
from MyTorch.operations import Flatten
from MyTorch.activations import ReLU, Softmax
from MyTorch.layers import Linear, Dropout


import keras
class TwoLayerModel(Model):
    def __init__(self):
        super().__init__()
        self.flatten = Flatten()
        self.linear1 = Linear(784, 128)
        self.linear2 = Linear(128, 10)
        self.dropout = Dropout(0.3)
        self.relu = ReLU()
        self.softmax = Softmax(dim=1)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout(x)
        x = self.softmax(x)
        return x
    

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)


In [2]:
from MyTorch.optimizers import SGD
from MyTorch.losses import MSE
from MyTorch import Tensor
import numpy as np

model = TwoLayerModel()
loss_fn = MSE()
optimizer = SGD(model.get_parameters(), lr=0.001)

batch_size = 128
epochs = 20

for epoch in range(epochs):
    model.train()
    for i in range(0, len(x_train), batch_size):
        batch_x = x_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        batch_x = Tensor(batch_x)
        batch_y = Tensor(batch_y)
        
        output = model(batch_x)
        
        loss = loss_fn(output, batch_y)
        
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
    model.eval()
    output = model(x_test)
    acc = np.sum(np.argmax(output.data, axis=1) == np.argmax(y_test, axis=1)) / len(x_test)
    print("Epoch: %d, Loss: %.5f, Accuracy: %.5f" % (epoch+1, loss.data, acc))

Epoch: 1, Loss: 0.06585, Accuracy: 0.90120
Epoch: 2, Loss: 0.06243, Accuracy: 0.92270
Epoch: 3, Loss: 0.06483, Accuracy: 0.92990
Epoch: 4, Loss: 0.06298, Accuracy: 0.93700
Epoch: 5, Loss: 0.06530, Accuracy: 0.94300
Epoch: 6, Loss: 0.06089, Accuracy: 0.94380
Epoch: 7, Loss: 0.05962, Accuracy: 0.94840
Epoch: 8, Loss: 0.07035, Accuracy: 0.95240
Epoch: 9, Loss: 0.06406, Accuracy: 0.95290
Epoch: 10, Loss: 0.06342, Accuracy: 0.95330
Epoch: 11, Loss: 0.05719, Accuracy: 0.95630
Epoch: 12, Loss: 0.05779, Accuracy: 0.95940
Epoch: 13, Loss: 0.05774, Accuracy: 0.95900
Epoch: 14, Loss: 0.05569, Accuracy: 0.96270
Epoch: 15, Loss: 0.05715, Accuracy: 0.96310
Epoch: 16, Loss: 0.06152, Accuracy: 0.96360
Epoch: 17, Loss: 0.05982, Accuracy: 0.96540
Epoch: 18, Loss: 0.05820, Accuracy: 0.96490
Epoch: 19, Loss: 0.06405, Accuracy: 0.96820
Epoch: 20, Loss: 0.06015, Accuracy: 0.96700


In [3]:
model.save('model.npz')

In [4]:
model = TwoLayerModel()
model.load('model.npz')

model.eval()
output = model(x_test)
acc = np.sum(np.argmax(output.data, axis=1) == np.argmax(y_test, axis=1)) / len(x_test)
print("Test Accuracy: %.5f" % (acc))

Test Accuracy: 0.96700
